In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%matplotlib inline
import torch
import numpy as np

knock90

In [3]:
!cat /etc/issue
!python -V
!nvcc -V
!dpkg -l | grep "cudnn"

Ubuntu 22.04.2 LTS \n \l

Python 3.10.6
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
hi  libcudnn8                                 8.9.0.131-1+cuda11.8                    amd64        cuDNN runtime libraries
ii  libcudnn8-dev                             8.9.0.131-1+cuda11.8                    amd64        cuDNN development libraries and headers


In [7]:
!tar -zxvf /content/drive/MyDrive/kftt-data-1.0.tar.gz
!tar -zxvf /content/drive/MyDrive/TensorRT-8.5.3.1.Ubuntu-20.04.aarch64-gnu.cuda-11.8.cudnn8.6.tar.gz

kftt-data-1.0/
kftt-data-1.0/data/
kftt-data-1.0/data/orig/
kftt-data-1.0/data/orig/kyoto-tune.en
kftt-data-1.0/data/orig/kyoto-dev.ja
kftt-data-1.0/data/orig/kyoto-dev.en
kftt-data-1.0/data/orig/kyoto-train.en
kftt-data-1.0/data/orig/kyoto-tune.ja
kftt-data-1.0/data/orig/kyoto-train.ja
kftt-data-1.0/data/orig/kyoto-test.ja
kftt-data-1.0/data/orig/kyoto-test.en
kftt-data-1.0/data/tok/
kftt-data-1.0/data/tok/kyoto-tune.en
kftt-data-1.0/data/tok/kyoto-dev.ja
kftt-data-1.0/data/tok/kyoto-train.cln.en
kftt-data-1.0/data/tok/kyoto-dev.en
kftt-data-1.0/data/tok/kyoto-train.en
kftt-data-1.0/data/tok/kyoto-tune.ja
kftt-data-1.0/data/tok/kyoto-train.cln.ja
kftt-data-1.0/data/tok/kyoto-train.ja
kftt-data-1.0/data/tok/kyoto-test.ja
kftt-data-1.0/data/tok/kyoto-test.en
kftt-data-1.0/README.txt
TensorRT-8.5.3.1/
TensorRT-8.5.3.1/doc/
TensorRT-8.5.3.1/doc/Acknowledgements.txt
TensorRT-8.5.3.1/doc/Readme.txt
TensorRT-8.5.3.1/graphsurgeon/
TensorRT-8.5.3.1/graphsurgeon/graphsurgeon-0.4.6-py2.py3-none-

In [9]:
!pip install spacy>=3.5.0
!pip install ginza

In [10]:
!cat kftt-data-1.0/data/orig/kyoto-train.ja | sed 's/\s+/ /g' | ginzame > train.ginza.ja
!cat kftt-data-1.0/data/orig/kyoto-dev.ja | sed 's/\s+/ /g' | ginzame > dev.ginza.ja
!cat kftt-data-1.0/data/orig/kyoto-test.ja | sed 's/\s+/ /g' | ginzame > test.ginza.ja

2023-07-24 06:30:34.258059: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 06:30:35.260068: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-24 06:30:36.644871: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-24 06:30:36.645437: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [11]:
for src, dst in [
  ('train.ginza.ja', 'train.spacy.ja'),
  ('dev.ginza.ja', 'dev.spacy.ja'),
  ('test.ginza.ja', 'test.spacy.ja'),
]:
  with open(src) as f:
    lst = []
    tmp = []
    for x in f:
      x = x.strip()
      if x == 'EOS':
        lst.append(' '.join(tmp))
        tmp = []
      elif x != '':
        tmp.append(x.split('\t')[0])
  with open(dst, 'w') as f:
    for line in lst:
      print(line, file=f)

In [12]:
import re
import spacy

nlp = spacy.load('en_core_web_sm')
for src, dst in [
  ('kftt-data-1.0/data/orig/kyoto-train.en', 'train.spacy.en'),
  ('kftt-data-1.0/data/orig/kyoto-dev.en', 'dev.spacy.en'),
  ('kftt-data-1.0/data/orig/kyoto-test.en', 'test.spacy.en'),
]:
  with open(src) as f, open(dst, 'w') as g:
    for x in f:
      x = x.strip()
      x = re.sub(r'\s+', ' ', x)
      x = nlp.make_doc(x)
      x = ' '.join([doc.text for doc in x])
      print(x, file=g)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


knock91

In [13]:
!pip install fairseq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 19.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.8/283.8 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11288385 sha256=8a52773cec5d4ac0f4deaac6fd260a9c4106d3655eb80ae825d515b600c0df08
  Stored in directory: /root/.cache/pip/wheels/e4/35/55/9c66f65ec7c83fd6fbc2b9502a0ac81b2448a1196159dacc32
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=1

In [14]:
!fairseq-preprocess -s ja -t en \
    --trainpref /content/train.spacy \
    --validpref /content/dev.spacy \
    --destdir /content/data91  \
    --thresholdsrc 5 \
    --thresholdtgt 5 \
    --workers 20

2023-07-24 06:43:13.216972: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-24 06:43:15 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-07-24 06:43:15 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_file=None, aim_repo=None, aim_run_hash=None, tensorboard_logdir=None, wandb_project=None, azureml_logging=False, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, on_cpu_convert_precision=False, min_loss_scale=0.0001, threshold_loss_scale=None, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, quantization_config_path=None, profile=False, reset_logging=False, s

In [ ]:
!fairseq-train data91 \
  --fp16 \
  --save-dir save91 \
  --max-epoch 10 \
  --arch transformer --share-decoder-input-output-embed \
  --optimizer adam --clip-norm 1.0 \
  --lr 1e-3 --lr-scheduler inverse_sqrt --warmup-updates 2000 \
  --update-freq 1 \
  --dropout 0.2 --weight-decay 0.0001 \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
  --max-tokens 8000 > 91.log

2023-07-24 06:48:31.719640: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
epoch 001:   0% 0/1822 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
epoch 001: 100% 1821/1822 [11:40<00:00,  2.75it/s, loss=7.535, nll_loss=6.241, ppl=75.63, wps=17760.1, ups=2.67, wpb=6640.9, bsz=220.1, num_updates=1800, lr=0.0009, gnorm=0.882, clip=22, loss_scale=4, train_wall=37, gb_free=6.3, wall=695]
epoch 001 | valid on 'valid' subset:   0% 0/8 [00:00<?, ?it/s]
epoch 001 | valid on 'valid' subset:  12% 1/8 [00:00<00:01,  5.45it/s]
epoch 001 | valid on 'valid' subset:  25% 2/8 [00:00<00:00,  7.28it/s]
epoch 001 | valid on 'valid' subset:  38% 3/8 [00:00<00:00,  8.22it/s]
epoch 001 | valid on 'valid' subset:  50% 4/8 [00:00<00:00,  8.66it/s]
epoch 001 | valid on 'valid' subset:  62% 5/8 [00:0

knock92

In [ ]:
!fairseq-interactive --path save91/checkpoint10.pt data91 < test.spacy.ja | grep '^H' | cut -f3 > 92.out

knock93

In [ ]:
!fairseq-score --sys 92.out --ref test.spacy.en

knock94

In [ ]:
%%bash

for N in `seq 1 20` ; do
    fairseq-interactive --path save91/checkpoint10.pt --beam $N data91 < test.spacy.ja | grep '^H' | cut -f3 > 94.$N.out
done

for N in `seq 1 20` ; do
    fairseq-score --sys 94.$N.out --ref test.spacy.en > 94.$N.score
done

In [ ]:
import matplotlib.pyplot as plt

def read_score(filename):
  with open(filename) as f:
    x = f.readlines()[1]
    x = re.search(r'(?<=BLEU4 = )\d*\.\d*(?=,)', x)
    return float(x.group())

xs = range(1, 21)
ys = [read_score(f'94.{x}.score') for x in xs]
plt.plot(xs, ys)
plt.show()

knock95

In [ ]:
import sentencepiece as spm

spm.SentencePieceTrainer.Train('--input=kftt-data-1.0/data/orig/kyoto-train.ja --model_prefix=kyoto_ja --vocab_size=16000 --character_coverage=1.0')

sp = spm.SentencePieceProcessor()
sp.Load('kyoto_ja.model')

for src, dst in [
  ('kftt-data-1.0/data/orig/kyoto-train.ja', 'train.sub.ja'),
  ('kftt-data-1.0/data/orig/kyoto-dev.ja', 'dev.sub.ja'),
  ('kftt-data-1.0/data/orig/kyoto-test.ja', 'test.sub.ja'),
  ]:
  with open(src) as f, open(dst, 'w') as g:
    for x in f:
      x = x.strip()
      x = re.sub(r'\s+', ' ', x)
      x = sp.encode_as_pieces(x)
      x = ' '.join(x)
      print(x, file=g)

In [ ]:
!subword-nmt learn-bpe -s 16000 < kftt-data-1.0/data/orig/kyoto-train.en > kyoto_en.codes
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/orig/kyoto-train.en > train.sub.en
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/orig/kyoto-dev.en > dev.sub.en
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/orig/kyoto-test.en > test.sub.en

In [ ]:
!fairseq-preprocess -s ja -t en \
    --trainpref train.sub \
    --validpref dev.sub \
    --destdir data95  \
    --workers 20

In [ ]:
!fairseq-train data95 \
    --fp16 \
    --save-dir save95 \
    --max-epoch 10 \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --clip-norm 1.0 \
    --lr 1e-3 --lr-scheduler inverse_sqrt --warmup-updates 2000 \
    --update-freq 1 \
    --dropout 0.2 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 8000 > 95.log

In [ ]:
!fairseq-interactive --path save95/checkpoint10.pt data95 < test.sub.ja | grep '^H' | cut -f3 | sed -r 's/(@@ )|(@@ ?$)//g' > 95.out

In [ ]:
def spacy_tokenize(src, dst):
  with open(src) as f, open(dst, 'w') as g:
    for x in f:
      x = x.strip()
      x = ' '.join([doc.text for doc in nlp(x)])
      print(x, file=g)
spacy_tokenize('95.out', '95.out.spacy')

In [ ]:
!fairseq-score --sys 95.out.spacy --ref test.spacy.en

In [ ]:
%%bash

for N in `seq 1 10` ; do
    fairseq-interactive --path save95/checkpoint10.pt --beam $N data95 < test.sub.ja | grep '^H' | cut -f3 | sed -r 's/(@@ )|(@@ ?$)//g' > 95.$N.out
done

In [ ]:
for i in range(1, 11):
  spacy_tokenize(f'95.{i}.out', f'95.{i}.out.spacy')

In [ ]:
%%bash

for N in `seq 1 10` ; do
    fairseq-score --sys 95.$N.out.spacy --ref test.spacy.en > 95.$N.score
done

In [ ]:
xs = range(1, 11)
ys = [read_score(f'95.{x}.score') for x in xs]
plt.plot(xs, ys)
plt.show()

knock96

In [ ]:
!fairseq-train data95 \
    --fp16 \
    --tensorboard-logdir log96 \
    --save-dir save96 \
    --max-epoch 5 \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --clip-norm 1.0 \
    --lr 1e-3 --lr-scheduler inverse_sqrt --warmup-updates 2000 \
    --dropout 0.2 --weight-decay 0.0001 \
    --update-freq 1 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 8000 > 96.log

knock97

In [ ]:
%%bash

fairseq-train data95 \
    --fp16 \
    --save-dir save97_1 \
    --max-epoch 10 \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --clip-norm 1.0 \
    --lr 1e-3 --lr-scheduler inverse_sqrt --warmup-updates 2000 \
    --dropout 0.1 --weight-decay 0.0001 \
    --update-freq 1 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 8000 > 97_1.log

fairseq-train data95 \
    --fp16 \
    --save-dir save97_3 \
    --max-epoch 10 \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --clip-norm 1.0 \
    --lr 1e-3 --lr-scheduler inverse_sqrt --warmup-updates 2000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --update-freq 1 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 8000 > 97_3.log

fairseq-train data95 \
    --fp16 \
    --save-dir save97_5 \
    --max-epoch 10 \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --clip-norm 1.0 \
    --lr 1e-3 --lr-scheduler inverse_sqrt --warmup-updates 2000 \
    --dropout 0.5 --weight-decay 0.0001 \
    --update-freq 1 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 8000 > 97_5.log

fairseq-interactive --path save97_1/checkpoint10.pt data95 < test.sub.ja | grep '^H' | cut -f3 | sed -r 's/(@@ )|(@@ ?$)//g' > 97_1.out
fairseq-interactive --path save97_3/checkpoint10.pt data95 < test.sub.ja | grep '^H' | cut -f3 | sed -r 's/(@@ )|(@@ ?$)//g' > 97_3.out
fairseq-interactive --path save97_5/checkpoint10.pt data95 < test.sub.ja | grep '^H' | cut -f3 | sed -r 's/(@@ )|(@@ ?$)//g' > 97_5.out

In [ ]:
spacy_tokenize('97_1.out', '97_1.out.spacy')
spacy_tokenize('97_3.out', '97_3.out.spacy')
spacy_tokenize('97_5.out', '97_5.out.spacy')

In [ ]:
!fairseq-score --sys 97_1.out.spacy --ref test.spacy.en
!fairseq-score --sys 97_3.out.spacy --ref test.spacy.en
!fairseq-score --sys 97_5.out.spacy --ref test.spacy.en

knock98

In [ ]:
import tarfile

with tarfile.open('en-ja.tar.gz') as tar:
  for f in tar.getmembers():
    if f.name.endswith('txt'):
      text = tar.extractfile(f).read().decode('utf-8')
      break

data = text.splitlines()
data = [x.split('\t') for x in data]
data = [x for x in data if len(x) == 4]
data = [[x[3], x[2]] for x in data]

with open('jparacrawl.ja', 'w') as f, open('jparacrawl.en', 'w') as g:
  for j, e in data:
    print(j, file=f)
    print(e, file=g)

In [ ]:
with open('jparacrawl.ja') as f, open('train.jparacrawl.ja', 'w') as g:
  for x in f:
    x = x.strip()
    x = re.sub(r'\s+', ' ', x)
    x = sp.encode_as_pieces(x)
    x = ' '.join(x)
    print(x, file=g)

In [ ]:
!subword-nmt apply-bpe -c kyoto_en.codes < jparacrawl.en > train.jparacrawl.en

In [ ]:
%%bash

fairseq-preprocess -s ja -t en \
    --trainpref train.jparacrawl \
    --validpref dev.sub \
    --destdir data98  \
    --workers 20

fairseq-train data98 \
    --fp16 \
    --save-dir save98_1 \
    --max-epoch 3 \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --clip-norm 1.0 \
    --lr 1e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.1 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 8000 > 98_1.log

fairseq-interactive --path save98_1/checkpoint3.pt data98 < test.sub.ja | grep '^H' | cut -f3 | sed -r 's/(@@ )|(@@ ?$)//g' > 98_1.out

In [ ]:
spacy_tokenize('98_1.out', '98_1.out.spacy')

In [ ]:
!fairseq-score --sys 98_1.out.spacy --ref test.spacy.en

In [ ]:
%%bash

fairseq-preprocess -s ja -t en \
    --trainpref train.sub \
    --validpref dev.sub \
    --tgtdict data98/dict.en.txt \
    --srcdict data98/dict.ja.txt \
    --destdir data98_2  \
    --workers 20

fairseq-train data98_2 \
    --fp16 \
    --restore-file save98_1/checkpoint3.pt \
    --save-dir save98_2 \
    --max-epoch 10 \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --clip-norm 1.0 \
    --lr 1e-3 --lr-scheduler inverse_sqrt --warmup-updates 2000 \
    --dropout 0.1 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 8000 > 98_2.log

fairseq-interactive --path save98_2/checkpoint10.pt data98_2 < test.sub.ja | grep '^H' | cut -f3 | sed -r 's/(@@ )|(@@ ?$)//g' > 98_2.out

In [ ]:
spacy_tokenize('98_2.out', '98_2.out.spacy')

In [ ]:
!fairseq-score --sys 98_2.out.spacy --ref test.spacy.en